In [1]:
import numpy as np

In [8]:
N = int(1e8)

def compute_pi(N):
    pts = np.random.uniform(0, 1, (2, N))
    M = np.count_nonzero((pts**2).sum(axis=0) < 1)
    return 4 * M / N

In [10]:
%timeit compute_pi(int(1e8))

1.89 s ± 50.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
